In [1]:
import pandas as pd
data = pd.read_csv("/kaggle/input/train-shuttle/train.csv")

In [2]:
data.columns

Index(['rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
       'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
       'landing_height', 'landing_area', 'landing_x', 'landing_y',
       'lose_reason', 'getpoint_player', 'player_location_area',
       'player_location_x', 'player_location_y', 'opponent_location_area',
       'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
       'rally_id', 'rally_length'],
      dtype='object')

In [3]:
s = data[data["ball_round"]==1]#, data["player"]==0]
s = s[s["player_location_area"]==8]
s

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,...,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length
0,1,1,00:06:00,10827,1,0,0,short service,0.0,1,...,8,170.8,308.95,8,236.7,675.21,1,1,0,8
8,8,1,00:09:20,16805,2,6,1,short service,0.0,1,...,8,186.0,630.55,8,132.6,284.99,1,1,7,5
13,9,1,00:09:44,17546,3,6,0,short service,0.0,1,...,8,165.3,309.01,8,238.1,687.10,1,1,8,24
37,11,1,00:10:58,19747,5,6,0,short service,0.0,1,...,8,174.3,341.99,8,238.5,669.18,1,1,10,10
47,12,1,00:11:33,20819,5,7,0,short service,0.0,1,...,8,185.3,308.77,8,124.6,672.64,1,1,11,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30112,32,1,00:49:28,89066,15,17,4,short service,0.0,1,...,8,188.0,323.59,8,128.4,634.91,2,58,4911,9
30121,33,1,00:50:02,90064,15,18,34,short service,0.0,1,...,8,165.9,645.69,8,228.6,301.39,2,58,4912,18
30139,35,1,00:51:55,93465,17,18,4,short service,0.0,1,...,8,171.2,297.32,8,225.1,646.86,2,58,4914,7
30146,38,1,00:54:16,97680,18,20,34,short service,0.0,1,...,8,183.8,621.68,8,136.7,287.97,2,58,4917,6


In [4]:
import numpy as np
weight = np.zeros(35)
i=0

while i<len(data):
    weight[data["player"][i+1]]+=1
    i+=data["rally_length"][i]
play = np.zeros(35)    
for i in range(35):
    play[i]=i
    weight[i]=1/weight[i]
weight   
        

array([0.01492537, 0.05555556, 0.00478469, 0.06666667, 0.01587302,
       0.0046729 , 0.03571429, 0.02325581, 0.02857143, 0.01724138,
       0.00675676, 0.01886792, 0.03448276, 0.01041667, 0.00641026,
       0.01030928, 0.0049505 , 0.0212766 , 0.03030303, 0.03225806,
       0.02702703, 0.01960784, 0.02564103, 0.02702703, 0.01923077,
       0.03030303, 0.02857143, 0.01639344, 0.02702703, 0.02702703,
       0.01666667, 0.01785714, 0.025     , 0.02564103, 0.08333333])

In [5]:
def filter(data,player1,player1_loc):
    out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
    output = pd.DataFrame(columns=out_col)     
    for i in range(len(data)):
        if data["player"][i]==player1 and data["player_location_area"][i]==player1_loc:
            output.loc[len(output), output.columns] = data.iloc[i] 
            #output= output.append(data.iloc[i])            
    
    return output


In [6]:
import math

def filter_loc(data,player1,loc_x, loc_y, error):
    out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
    output = pd.DataFrame(columns=out_col)
    loc = [loc_x,loc_y]
    for i in range(len(data)):
        if data["player"][i]==player1:
            data_loc = [float(data["player_location_x"][i]),float(data["player_location_y"][i])]
            if math.dist(loc, data_loc) <= error:
                output.loc[len(output), output.columns] = data.iloc[i] 
            #output= output.append(data.iloc[i])            
    
    return output

In [7]:
filter_loc(data,6,275,575, 20)

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,...,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length


In [8]:
from random import choices
out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
augm = pd.DataFrame(columns=out_col)

In [9]:
import numpy as np
import random
def augmen_rally(data_in,playera, playerb,rally_id,match_id,se,data_out,num_of_rally):
    rally_len=1
    total_rally=0
    loc = random.randint(7,8)
    out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
    output = pd.DataFrame(columns=out_col) 
    
    loc = random.randint(1,10)   
    while total_rally<num_of_rally:
        if rally_len%2 :
            player = playera           
        else:
            player = playerb
        
        filt = filter(data_in,player,loc)
        if len(filt)>0:
            sel = random.randint(0,len(filt)-1)
        else:
            for i in range(rally_len):
                data_out.loc[len(data_out)-i-1,['rally_length']] = rally_len
            if (rally_len+1)%2:
                data_out["getpoint_player"][len(data_out)-1] = playerb
            else :
                data_out["getpoint_player"][len(data_out)-1] = playera
            data_out["lose_reason"][len(data_out)-1] = "no such data"             
            total_rally+=1
            continue
        shoot = filt.iloc[sel]
        shoot["rally"] = rally_id+1+total_rally
        shoot["ball_round"] = rally_len
        shoot["player"] = player
        loc = shoot["opponent_location_area"]
        shoot["set"] = se
        shoot["match_id"] = match_id
        shoot["rally_id"] = rally_id+total_rally
        
        if len(str(shoot["lose_reason"]))>3 or str(shoot["lose_reason"])=="out":
            if shoot["getpoint_player"]!=player:
                if player == playera:
                    shoot["getpoint_player"]=playerb
                else:
                    shoot["getpoint_player"]=playera
            data_out.loc[len(data_out), data_out.columns] = shoot
            for i in range(rally_len):
                data_out.loc[len(data_out)-i-1,['rally_length']] = rally_len
                #data_out = data_out.assign(rally_length=rally_len)
            total_rally+=1
            rally_len = 1
            loc = random.randint(1,10)
            continue
        rally_len+=1
        data_out.loc[len(data_out), data_out.columns] = shoot 
        #output = output.append(shoot)
    return data_out
    
    

In [10]:
import random
def augmen_rally_loc(data_in,playera, playerb,rally_id,
                     match_id,se,data_out,num_of_rally,error):
    rally_len=1
    total_rally=0
    loc = random.randint(7,8)
    out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
    output = pd.DataFrame(columns=out_col)  
    filt = filter(data_in,playera,loc)   
    sel = random.randint(0,len(filt)-1)
    shoot = filt.iloc[sel]
    loc_x = shoot["opponent_location_x"]
    loc_y = shoot["opponent_location_y"]
    shoot["rally"] = rally_id+1+total_rally
    shoot["ball_round"] = rally_len
    shoot["player"] = playera    
    shoot["set"] = se
    shoot["match_id"] = match_id
    shoot["rally_id"] = rally_id+total_rally
    data_out.loc[len(data_out), data_out.columns] = shoot
    while total_rally<=num_of_rally:
        if rally_len >= 70:
            for i in range(rally_len):
                data_out.loc[len(data_out)-i-1,['rally_length']] = rally_len
            if (rally_len+1)%2:
                data_out["getpoint_player"][len(data_out)-1] = playerb
            else :
                data_out["getpoint_player"][len(data_out)-1] = playera
            data_out["lose_reason"][len(data_out)-1] = "out of legnth"
            rall_len=1
            loc = random.randint(7,8)
            filt = filter(data_in,playera,loc)   
            sel = random.randint(0,len(filt)-1)
            shoot = filt.iloc[sel]
            loc_x = shoot["opponent_location_x"]
            loc_y = shoot["opponent_location_y"]
            shoot["rally"] = rally_id+1+total_rally
            shoot["ball_round"] = rally_len
            shoot["player"] = playera    
            shoot["set"] = se
            shoot["match_id"] = match_id
            shoot["rally_id"] = rally_id+total_rally
            data_out.loc[len(data_out), data_out.columns] = shoot
            total_rally+=1
            continue
        if (rally_len+1)%2 :
            player = playera           
        else:
            player = playerb
        
        filt = filter_loc(data_in,player,loc_x,loc_y,error)
        if len(filt)>0:
            sel = random.randint(0,len(filt)-1)
        else:
            #filt = filter_loc(data_in,player,loc_x,loc_y,error+2)
            for i in range(rally_len):
                data_out.loc[len(data_out)-i-1,['rally_length']] = rally_len
            if (rally_len+1)%2:
                data_out["getpoint_player"][len(data_out)-1] = playerb
            else :
                data_out["getpoint_player"][len(data_out)-1] = playera
            data_out["lose_reason"][len(data_out)-1] = "no such data"
            rall_len=1
            loc = random.randint(7,8)
            filt = filter(data_in,playera,loc)   
            sel = random.randint(0,len(filt)-1)
            shoot = filt.iloc[sel]
            loc_x = shoot["opponent_location_x"]
            loc_y = shoot["opponent_location_y"]
            shoot["rally"] = rally_id+1+total_rally
            shoot["ball_round"] = rally_len
            shoot["player"] = playera    
            shoot["set"] = se
            shoot["match_id"] = match_id
            shoot["rally_id"] = rally_id+total_rally
            data_out.loc[len(data_out), data_out.columns] = shoot
            total_rally+=1
            continue
            
        shoot = filt.iloc[sel]
        shoot["rally"] = rally_id+1+total_rally
        shoot["ball_round"] = rally_len
        shoot["player"] = player
        loc_x = shoot["opponent_location_x"]
        loc_y = shoot["opponent_location_y"]
        shoot["set"] = se
        shoot["match_id"] = match_id
        shoot["rally_id"] = rally_id+total_rally
        
        if len(str(shoot["lose_reason"]))>3 or str(shoot["lose_reason"])=="out":
            if shoot["getpoint_player"]!=player:
                if player == playera:
                    shoot["getpoint_player"]=playerb
                else:
                    shoot["getpoint_player"]=playera
            data_out.loc[len(data_out), data_out.columns] = shoot
            for i in range(rally_len):
                data_out.loc[len(data_out)-i-1,['rally_length']] = rally_len
                #data_out = data_out.assign(rally_length=rally_len)
            total_rally+=1
            rally_len = 1
            loc = random.randint(1,10)
            continue
        rally_len+=1
        data_out.loc[len(data_out), data_out.columns] = shoot 
        #output = output.append(shoot)
    return data_out

In [11]:
d=augmen_rally_loc(data,0, 1,0,0,0,augm,10,24)

In [12]:
d

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,...,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length
0,1,1,00:48:16,86906,19,12,0,net shot,0.0,0,...,7,140.3,401.04,8,182.9,633.0,0,0,0,7
1,1,1,00:18:43,33709,10,12,1,short service,0.0,1,...,8,175.5,638.54,8,135.2,259.85,0,0,0,4
2,1,2,00:18:46,33782,10,12,0,defensive shot,0.0,0,...,8,132.2,264.96,9,111.5,771.59,0,0,0,4
3,1,3,00:18:45,33768,10,12,1,smash,0.0,0,...,9,131.5,759.81,8,152.3,282.26,0,0,0,4
4,1,4,00:48:39,87577,20,12,0,drive,0.0,1,...,8,162.1,304.13,8,214.6,678.64,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,11,39,00:44:14,79624,12,7,1,defensive shot,0.0,1,...,8,180.6,291.8,7,188.7,568.83,0,0,10,43
87,11,40,00:42:29,76482,11,6,0,lob,0.0,0,...,7,172.7,554.88,8,171.4,311.34,0,0,10,43
88,11,41,00:44:14,79624,12,7,1,defensive shot,0.0,1,...,8,180.6,291.8,7,188.7,568.83,0,0,10,43
89,11,42,00:19:46,35597,12,12,0,lob,0.0,0,...,7,171.4,573.6,7,160.7,419.91,0,0,10,43


In [13]:
import datetime;
 
# ct stores current time

out_col =[
        'rally', 'ball_round', 'time', 'frame_num', 'roundscore_A',
        'roundscore_B', 'player', 'type', 'aroundhead', 'backhand',
        'landing_height', 'landing_area', 'landing_x', 'landing_y',
        'lose_reason', 'getpoint_player', 'player_location_area',
        'player_location_x', 'player_location_y', 'opponent_location_area',
        'opponent_location_x', 'opponent_location_y', 'set', 'match_id',
        'rally_id', 'rally_length'
     ]
output = pd.DataFrame(columns=out_col)

def finall_genorator(data_in,data_out,rally_num,start_rally_id):
    match=0
    while match < rally_num:
        i = random.choices(play,weight)
        j = random.choices(play,weight)    
        if(i==j):
            continue
        if match%10==0:
            ct = datetime.datetime.now()
            
            print("matching:",match," current time:-", ct)
            print("----------------------------------------------")
        #data_out = augmen_rally_loc(data_in,int(i[0]), int(j[0]),start_rally_id+match,
                                    #0,0,data_out,1,25)
        data_out = augmen_rally(data_in,int(i[0]), int(j[0]),start_rally_id+match,
                                    0,0,data_out,1)
        match+=1
    return data_out
            
       
        

In [14]:
d = finall_genorator(data,output,800,4920)

matching: 0  current time:- 2023-06-10 14:35:00.597880
----------------------------------------------
matching: 10  current time:- 2023-06-10 14:36:09.601343
----------------------------------------------
matching: 20  current time:- 2023-06-10 14:36:39.463108
----------------------------------------------
matching: 30  current time:- 2023-06-10 14:37:35.612725
----------------------------------------------
matching: 40  current time:- 2023-06-10 14:38:29.139462
----------------------------------------------
matching: 50  current time:- 2023-06-10 14:39:14.888950
----------------------------------------------
matching: 60  current time:- 2023-06-10 14:40:14.434362
----------------------------------------------
matching: 70  current time:- 2023-06-10 14:41:25.587026
----------------------------------------------
matching: 80  current time:- 2023-06-10 14:41:46.675028
----------------------------------------------
matching: 90  current time:- 2023-06-10 14:42:53.927288
------------------

In [15]:
d

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,...,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length
0,4921,1,00:47:33,85608,3,3,26,drop,0.0,0,...,3,252.5,778.29,8,191.9,292.8,0,0,4920,15
1,4921,2,01:07:36,121687,12,5,34,defensive shot,0.0,0,...,8,198.3,685.05,8,246.5,224.85,0,0,4920,15
2,4921,3,00:23:39,42582,13,12,26,net shot,0.0,1,...,8,130.9,354.07,8,166.4,654.38,0,0,4920,15
3,4921,4,01:07:49,122088,12,5,34,drop,0.0,0,...,8,242.6,737.44,8,212.3,241.17,0,0,4920,15
4,4921,5,00:52:00,93618,9,4,26,smash,0.0,0,...,8,150.3,744.51,8,136.8,311.38,0,0,4920,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8369,5719,3,00:36:22,65482,9,17,3,short service,0.0,1,...,8,188.6,311.64,8,125.7,643.5,0,0,5718,6
8370,5719,4,00:21:46,39200,13,11,34,net shot,0.0,1,...,8,242.9,339.31,8,196.9,730.79,0,0,5718,6
8371,5719,5,00:08:18,14961,4,1,3,push/rush,0.0,1,...,8,198.1,630.14,8,193.5,254.93,0,0,5718,6
8372,5719,6,00:20:59,37782,12,10,34,push/rush,0.0,0,...,8,199.3,299.37,8,206.4,595.09,0,0,5718,6


In [16]:
d.to_csv("augment_databyarea.csv")